# 第八个实例：文本的特征提取

## 1 One-hot编码

One-hot编码也称之为独热编码，是自然语言处理(NLP)里常用的文本特征提取方法。

导入库：

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

定义输出形式：

In [2]:
def onehot_encode(x):          #输入为一个列表，如x=['体育', '军事', '娱乐', '教育', '文化']
    ohe = OneHotEncoder()     #onehot编码方法
    ind = list(range(len(x)))      #获得索引列表
    ohe.fit(np.array([ind]).T)    #数据转成列向量，然后fit是看可以取多少个值
    x_processed = ohe.transform(np.array([ind]).T).toarray()  #然后转成one-hot编码，再转成数组
    label_dict ={}          #构建键值为对象及对应one-hot列表的字典
    label_id_dict={}        #构建键值为对象及对应索引的字典
    for index in ind:
        if x[index] not in label_dict.keys():
            label_dict[x[index]] = x_processed[index].tolist()   #添加键值对，onehot编码为列表形式
            #每个数据对应一个id，该id即为one-hot编码1所在的位置
            for order in range(len(label_dict[x[index]])):
                if label_dict[x[index]][order] == 1:
                    label_id_dict[x[index]] = order
    
    return label_dict,label_id_dict


运行：

In [3]:
x=['体育', '军事', '娱乐', '教育', '文化', '时尚', '科技', '财经']
onehot_encode(x)

({'体育': [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  '军事': [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  '娱乐': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  '教育': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  '文化': [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
  '时尚': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  '科技': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  '财经': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]},
 {'体育': 0, '军事': 1, '娱乐': 2, '教育': 3, '文化': 4, '时尚': 5, '科技': 6, '财经': 7})

从结果看one-hot编码的特点：
1.	One-hot向量存在稀疏性（sparse），向量中大部分元素是0
2.	每一个向量相互垂直（orthogonal representations），词之间相互独立
3.	无法表达相关词背后的语义（semantically weak）


有关one-hot编码的具体使用方法参见https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder

# 2 词向量与word2vec

    1)Word2vec是Google在2013年开源的一款用于词向量计算的工具，可以在百万数量级的词典和上亿的数据集上进行高效地训练。
    2)该工具训练得到的词向量（word embedding），可以很好地度量词与词之间的相似性。
    3)Word2vec采用浅层神经网络模型：CBOW模型和Skip-gram模型。

### 准备工作

    1）安装gensim包：pip install gensim
    2）安装jieba包：conda install jieba
    3）安装繁体字转简体字包：pip install zhconv 

导入库

In [1]:
from gensim.corpora import WikiCorpus    #导入Wiki语料库
from gensim.models import word2vec      #导入word2vec模型
import zhconv                          #导入zhconv模块
import jieba                           #导入jieba分词模块
import re                              #导入正则表达式模块
import multiprocessing                 #导入多进程模块

加载数据并提取内容

In [6]:
input_file_name = 'zhwiki-latest-pages-articles.xml.bz2'  #数据集名称，有后缀
output_file_name = 'corpus_cn.txt'                   #输出文件的名称
#加载数据


input_file = WikiCorpus(input_file_name, dictionary={})

with open(output_file_name, 'w', encoding="utf8") as output_file:
    print("开始处理数据")
    count = 0
#使用WikiCorpus类中的get_texts()方法读取文件，每篇文章转换为一行文本，并去掉标签符号等内容
    for text in input_file.get_texts():           
        output_file.write(' '.join(text) + '\n')
        count = count + 1
        if count % 10000 == 0:
            print('已处理%d条数据' % count)            
print('处理完成！')
#查看处理结果
with open('corpus_cn.txt',"r",encoding="utf8") as f:
    print(f.readlines()[:1])

开始处理数据


ParseError: not well-formed (invalid token): line 1762523, column 4 (<string>)

繁体字转简体字

In [7]:
input_file_name = 'corpus_cn.txt'
output_file_name = 'corpus_cn_simple.txt'
#读取文件
with open(input_file_name, 'r', encoding='utf-8') as input_file:
    lines = input_file.readlines()         #逐行读取文本内容
    count = 0
    with open(output_file_name, 'w', encoding='utf-8') as output_file:
  
        for line in lines:
            #繁体转简体，参数“zh-hans”
            output_file.write(zhconv.convert(line, 'zh-hans'))
            #每10000条数据输出一次结果
            count += 1
            if count % 10000 == 0:
                print('已转换%d条数据' % count)
print('处理完成！')
#查看结果
with open('corpus_cn_simple.txt',"r",encoding="utf8") as f:
    print(f.readlines()[:1])

处理完成！
['欧几里得 西元前三世纪的古希腊数学家 现在被认为是几何之父 此画为拉斐尔的作品 雅典学院 数学 是利用符号语言研究数量 结构 变化以及空间等概念的一门学科 从某种角度看属于形式科学的一种 数学透过抽象化和逻辑推理的使用 由计数 计算 数学家们拓展这些概念 对数学基本概念的完善 早在古埃及 而在古希腊那里有更为严谨的处理 从那时开始 数学的发展便持续不断地小幅进展 世纪的文艺复兴时期 致使数学的加速发展 直至今日 今日 数学使用在不同的领域中 包括科学 工程 医学 经济学和金融学等 有时亦会激起新的数学发现 并导致全新学科的发展 数学家也研究纯数学 就是数学本身的实质性内容 而不以任何实际应用为目标 虽然许多研究以纯数学开始 但其过程中也发现许多应用之处 词源 西方语言中 数学 一词源自于古希腊语的 其有 学习 学问 科学 数学研究 即使在其语源内 其形容词 意思为 和学习有关的 用功的 亦会被用来指 数学的 其在英语中表面上的复数形式 及在法语中的表面复数形式 可溯至拉丁文的中性复数 由西塞罗译自希腊文复数 此一希腊语被亚里士多德拿来指 万物皆数 的概念 汉字表示的 数学 一词大约产生于中国宋元时期 多指象数之学 但有时也含有今天上的数学意义 例如 秦九韶的 数学九章 永乐大典 数书九章 也被宋代周密所著的 癸辛杂识 记为 数学大略 数学通轨 明代柯尚迁著 数学钥 清代杜知耕著 数学拾遗 清代丁取忠撰 直到 经过中国数学名词审查委员会研究 算学 数学 两词的使用状况后 确认以 数学 表示今天意义上的数学含义 历史 奇普 印加帝国时所使用的计数工具 玛雅数字 数学有着久远的历史 中国古代的六艺之一就有 数学一词在西方有希腊语词源 mathematikós 意思是 学问的基础 源于 máthema 科学 知识 学问 时间的长短等抽象的数量关系 比如时间单位有日 季节和年等 算术 加减乘除 也自然而然地产生了 历史上曾有过许多不同的记数系统 在最初有历史记录的时候 为了解数字间的关系 为了测量土地 以及为了预测天文事件而形成的 结构 空间及时间方面的研究 到了 世纪 算术 微积分的概念也在此时形成 随着数学转向形式化 从古至今 数学便一直不断地延展 且与科学有丰富的相互作用 两者的发展都受惠于彼此 在历史上有著许多数学发现 并且直至今日都不断地有新的发现 

分词

In [8]:
input_file_name = 'corpus_cn_simple.txt'
output_file_name = 'corpus_cn_simple_separate.txt'
#读取文件
with open(input_file_name, 'r', encoding='utf-8') as input_file:   
    lines = input_file.readlines()
    count = 0
    with open(output_file_name, 'w', encoding='utf-8') as output_file:
        for line in lines:
            # jieba分词的结果是一个list，需要拼接，但是jieba把空格回车都当成一个字符处理
            output_file.write(' '.join(jieba.cut(line.split('\n')[0].replace(' ', ''))) + '\n')
            count += 1
            if count % 10000 == 0:
                print('已分词%d条数据' % count)
    print('处理完成！')
#查看结果
with open('corpus_cn_simple_separate.txt',"r",encoding="utf8") as f:
    print(f.readlines()[:1])

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\yan\AppData\Local\Temp\jieba.cache
Loading model cost 0.604 seconds.
Prefix dict has been built successfully.


处理完成！
['欧几里得 西元前 三 世纪 的 古希腊 数学家 现在 被 认为 是 几何 之父 此画 为 拉斐尔 的 作品 雅典 学院 数学 是 利用 符号语言 研究 数量 结构 变化 以及 空间 等 概念 的 一门 学科 从 某种 角度看 属于 形式 科学 的 一种 数学 透过 抽象化 和 逻辑推理 的 使用 由 计数 计算 数学家 们 拓展 这些 概念 对 数学 基本概念 的 完善 早 在 古埃及 而 在 古希腊 那里 有 更为 严谨 的 处理 从 那时 开始 数学 的 发展 便 持续 不断 地 小幅 进展 世纪 的 文艺复兴 时期 致使 数学 的 加速 发展 直至 今日 今日 数学 使用 在 不同 的 领域 中 包括 科学 工程 医学 经济学 和 金融学 等 有时 亦 会 激起 新 的 数学 发现 并 导致 全新 学科 的 发展 数学家 也 研究 纯数学 就是 数学 本身 的 实质性 内容 而 不以 任何 实际 应用 为 目标 虽然 许多 研究 以 纯数学 开始 但 其 过程 中 也 发现 许多 应用 之 处 词源 西方 语言 中 数学 一 词源 自于 古希腊 语 的 其 有 学习 学问 科学 数学 研究 即使 在 其 语源 内其 形容词 意思 为 和 学习 有关 的 用功 的 亦 会 被 用来 指 数学 的 其 在 英语 中 表面 上 的 复数 形式 及 在 法语 中 的 表面 复数 形式 可溯 至 拉丁文 的 中性 复数 由 西塞罗 译自 希腊文 复数 此一 希腊语 被 亚里士多德 拿来 指 万物 皆 数 的 概念 汉字 表示 的 数学 一词 大约 产生 于 中国 宋元 时期 多指 象数 之学 但 有时 也 含有 今天 上 的 数学 意义 例如 秦九韶 的 数学 九章 永乐 大典 数书 九章 也 被 宋代 周密 所著 的 癸 辛杂 识记 为 数学 大略 数学 通轨 明代 柯尚 迁著 数学 钥 清代 杜知 耕著 数学 拾遗 清代 丁取忠 撰 直到 经过 中国 数学 名词 审查 委员会 研究 算学 数学 两词 的 使用 状况 后 确认 以 数学 表示 今天 意义 上 的 数学 含义 历史 奇普 印加帝国 时所 使用 的 计数 工具 玛雅 数字 数学 有着 久远 的 历史 中国 古代 的 六艺 之一 就 有 数学 一词 在 西方 有 希腊语 词源 mathema

去除非中文词

In [9]:
# coding:utf-8
input_file_name = 'corpus_cn_simple_separate.txt'
output_file_name = 'corpus.txt'
#读取文件
with open(input_file_name, 'r', encoding='utf-8') as input_file:  
   
    lines = input_file.readlines()    
    count = 1
    cn_reg = '^[\u4e00-\u9fa5]+$'  #符合中文的正则表达式，^：字符串开始位置； $：字符串结尾位置
    
    with open(output_file_name, 'w', encoding='utf-8') as output_file:
        
        for line in lines:
            line_list = line.split('\n')[0].split(' ')
            line_list_new = []
            
            for word in line_list:
                if re.search(cn_reg, word):
                    line_list_new.append(word) 
                    
            output_file.write(' '.join(line_list_new) + '\n')
            count += 1
            if count % 10000 == 0:
                print('已处理%d条数据' % count)
    print("处理完成！")
#查看结果
with open('corpus.txt',"r",encoding="utf8") as f:
    print(f.readlines()[:1])

处理完成！
['欧几里得 西元前 三 世纪 的 古希腊 数学家 现在 被 认为 是 几何 之父 此画 为 拉斐尔 的 作品 雅典 学院 数学 是 利用 符号语言 研究 数量 结构 变化 以及 空间 等 概念 的 一门 学科 从 某种 角度看 属于 形式 科学 的 一种 数学 透过 抽象化 和 逻辑推理 的 使用 由 计数 计算 数学家 们 拓展 这些 概念 对 数学 基本概念 的 完善 早 在 古埃及 而 在 古希腊 那里 有 更为 严谨 的 处理 从 那时 开始 数学 的 发展 便 持续 不断 地 小幅 进展 世纪 的 文艺复兴 时期 致使 数学 的 加速 发展 直至 今日 今日 数学 使用 在 不同 的 领域 中 包括 科学 工程 医学 经济学 和 金融学 等 有时 亦 会 激起 新 的 数学 发现 并 导致 全新 学科 的 发展 数学家 也 研究 纯数学 就是 数学 本身 的 实质性 内容 而 不以 任何 实际 应用 为 目标 虽然 许多 研究 以 纯数学 开始 但 其 过程 中 也 发现 许多 应用 之 处 词源 西方 语言 中 数学 一 词源 自于 古希腊 语 的 其 有 学习 学问 科学 数学 研究 即使 在 其 语源 内其 形容词 意思 为 和 学习 有关 的 用功 的 亦 会 被 用来 指 数学 的 其 在 英语 中 表面 上 的 复数 形式 及 在 法语 中 的 表面 复数 形式 可溯 至 拉丁文 的 中性 复数 由 西塞罗 译自 希腊文 复数 此一 希腊语 被 亚里士多德 拿来 指 万物 皆 数 的 概念 汉字 表示 的 数学 一词 大约 产生 于 中国 宋元 时期 多指 象数 之学 但 有时 也 含有 今天 上 的 数学 意义 例如 秦九韶 的 数学 九章 永乐 大典 数书 九章 也 被 宋代 周密 所著 的 癸 辛杂 识记 为 数学 大略 数学 通轨 明代 柯尚 迁著 数学 钥 清代 杜知 耕著 数学 拾遗 清代 丁取忠 撰 直到 经过 中国 数学 名词 审查 委员会 研究 算学 数学 两词 的 使用 状况 后 确认 以 数学 表示 今天 意义 上 的 数学 含义 历史 奇普 印加帝国 时所 使用 的 计数 工具 玛雅 数字 数学 有着 久远 的 历史 中国 古代 的 六艺 之一 就 有 数学 一词 在 西方 有 希腊语 词源 意思 是 学问

训练词向量

In [11]:
input_file_name = 'corpus.txt'
model_file_name = 'wordvec.model'

sentences = word2vec.LineSentence(input_file_name)
#word2vec模型参量的设置
model = word2vec.Word2Vec(sentences,
            vector_size=300,                  # 词向量长度为300
            window=5,                 #表示当前词与预测词在一个句子中的最大距离是多少
            min_count=5,
            sg=0,                     #1是skip-gram，0是CBOW
            hs=0,                     #1是hierarchical-softmax，0是negative sampling。
                                      # hierarchical-softmax本质是把 N 分类问题变成 log(N)次二分类 
                                      # negative sampling本质是预测总体类别的一个子集
                                      # 二者均属于模型的训练技巧
            negative=5,                # 负样例的个数
            workers=multiprocessing.cpu_count())   #使用多线程进行处理

model.save(model_file_name)             #保存模型
print("训练模型结束...")

训练模型结束...


加载模型并测试效果

In [12]:
model_path = "wordvec.model"
wordvec = word2vec.Word2Vec.load(model_path)
#获得"华为"的词向量
wordvec.wv.get_vector("华为")

array([ 0.34529594,  0.24179251,  0.10473288,  0.1813133 , -0.00187204,
       -0.47839245, -0.25414222,  0.39364925, -0.63092554, -0.17613098,
       -0.05486733, -0.01311643, -0.04965291,  0.17734574,  0.10258082,
       -0.16858317, -0.10216668,  0.28379068, -0.21361233, -0.20631056,
       -0.25923428, -0.3837197 ,  0.29093954,  0.04620251,  0.14617884,
        0.3421932 , -0.23460528,  0.1324716 ,  0.13800356, -0.13280304,
       -0.0747147 ,  0.05705162,  0.10455812,  0.04380939,  0.24632953,
       -0.02389245,  0.20322406, -0.5009905 ,  0.07769388, -0.20932104,
       -0.04160252,  0.07472046,  0.18336317,  0.07800818,  0.14855577,
        0.15009408, -0.36532652, -0.11441483,  0.00242643,  0.45595047,
       -0.17194122, -0.03342921, -0.11724968,  0.07016382,  0.23549424,
        0.17653415,  0.06583297, -0.37028906, -0.02603964, -0.00109422,
        0.06381576, -0.22226156,  0.12218507,  0.1448392 , -0.1762116 ,
       -0.256432  ,  0.18719646,  0.2517517 ,  0.10768884,  0.16

In [13]:
#获得与"华为"最相近的5个词
wordvec.wv.most_similar("华为",topn=5)

[('中国移动', 0.9154809713363647),
 ('联想集团', 0.8963241577148438),
 ('雅虎', 0.8441826701164246),
 ('持股', 0.8399478197097778),
 ('日立', 0.8335539102554321)]

有关word2vec的具体使用方法参见https://radimrehurek.com/gensim/models/word2vec.html